<a href="https://colab.research.google.com/github/jfb10d/ml-practice/blob/main/notebooks/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-07-15 12:47:48--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  35.0MB/s    in 0.7s    

2025-07-15 12:47:49 (35.0 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code

In [ ]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
df_ratings['user'].value_counts()

,count
user,
11676,13602
198711,7550
153662,6109
98391,5891
35859,5850
...,...
119573,1
276706,1
276697,1


In [ ]:
len(df_ratings)

1149780

In [ ]:
user_remove_threshold = 200#np.ceil(df_ratings['user'].value_counts().quantile(0.99))
ratings_count = df_ratings['user'].value_counts()
keep_user = ratings_count[ratings_count >= user_remove_threshold].index
df_ratings_keep = df_ratings[df_ratings.user.isin(keep_user)]
len(df_ratings_keep)

527556

In [ ]:
book_remove_threshold = 100#np.ceil(df_ratings['isbn'].value_counts().quantile(0.99))
book_count = df_ratings['isbn'].value_counts()
keep_isbn = book_count[book_count >= book_remove_threshold].index
df_ratings_keep = df_ratings_keep[df_ratings_keep.isbn.isin(keep_isbn)]
len(df)

49781

In [ ]:
df_books = df_books.dropna()
df_books.isnull().sum()

,0
isbn,0
title,0
author,0


In [ ]:
df_ratings_keep.isnull().sum()

,0
user,0
isbn,0
rating,0


In [ ]:
# there are books that could be sharing the same title
books = ["Where the Heart Is (Oprah's Book Club (Paperback))", "I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
for book in books:
  if book in df_books.title.values:
    print(df_books[df_books.title == book].isbn.value_counts())

isbn
0446672211    1
Name: count, dtype: int64
isbn
0671888587    1
0553567187    1
Name: count, dtype: int64
isbn
0316782505    1
Name: count, dtype: int64
isbn
0345447840    1
0345447832    1
0451180240    1
0749902698    1
Name: count, dtype: int64
isbn
0060987561    1
0061097640    1
0965605914    1
0694519405    1
Name: count, dtype: int64


In [ ]:
df = df_ratings_keep.pivot(index='isbn', columns='user', values='rating').fillna(0)
df.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
isbn,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
0060008032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060096195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060173289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.index = df.join(df_books.set_index('isbn'))['title']
df = df.sort_index()
df.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model = NearestNeighbors(metric='cosine')
model.fit(df.values)
title = "Where the Heart Is (Oprah's Book Club (Paperback))"
distance, indice = model.kneighbors([df.loc[title].values], n_neighbors=6)
print(distance[0], df.iloc[indice[0]])

[0.         0.7234864  0.7677075  0.7699411  0.77085835 0.8016211 ] user                                                254     2276    2766    \
title                                                                        
Where the Heart Is (Oprah's Book Club (Paperback))     0.0     0.0     0.0   
The Lovely Bones: A Novel                              0.0     0.0     9.0   
I Know This Much Is True                               0.0     0.0     0.0   
The Surgeon                                            0.0     0.0     0.0   
The Weight of Water                                    0.0     0.0     0.0   
I'll Be Seeing You                                     0.0     0.0     0.0   

user                                                2977    3363    4017    \
title                                                                        
Where the Heart Is (Oprah's Book Club (Paperback))     0.0    10.0     0.0   
The Lovely Bones: A Novel                              0.0     0.0    10.

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  if book not in df.index:
    raise ValueError(f'book "{book}" is not in the dataframe!')
  if not book:
    raise ValueError('book title is empty')

  distance, indice = model.kneighbors([df.loc[book].values], n_neighbors=6)
  recommend_df = pd.DataFrame({
      'title'   : df.iloc[indice[0][::-1]].index.values,
      'distance': distance[0][::-1]
    })
  recommended_books = [book, recommend_df.values]


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074],
       ['The Lovely Bones: A Novel', 0.7234864234924316],
       ["Where the Heart Is (Oprah's Book Club (Paperback))", 0.0]],
      dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
